In [1]:
import os

In [2]:
base = 'data/stage1_test'
dirs = os.listdir(base)

In [3]:
os.listdir(os.path.join(base, dirs[0]))

['images']

In [4]:
id_to_image = {}
for el in dirs:
    image_dir = os.path.join(base, el, 'images')
    if not os.path.exists(image_dir):
        continue
    image_id = el
    images = os.listdir(image_dir)
    images = [os.path.join(image_dir, image) for image in images]

    id_to_image[image_id] = {}
    id_to_image[image_id]['images'] = images
    

In [5]:
id_to_image[dirs[0]]['images']`

['data/stage1_test/0114f484a16c152baa2d82fdd43740880a762c93f436c8988ac461c5c9dbe7d5/images/0114f484a16c152baa2d82fdd43740880a762c93f436c8988ac461c5c9dbe7d5.png']

In [11]:
print (len([key for key in id_to_image.keys()]))

65


In [6]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

# Darkness


In [7]:
import sklearn
from sklearn import mixture
from sklearn import cluster
class Example:
    def __init__(self, image_path):
        self.image_path = image_path
        self.image_id = image_path.split('/')[-1].split('.')[0]
        self.image = mpimg.imread(image_path)
        self.image_shape = self.image.shape
        self.gray_image = self.rgb2gray(self.image)

    def rgb2gray(self, rgb):
        return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])
    
    def combine_masks(self, masks):
        return np.sum(np.array([ mpimg.imread(mask) for mask in masks]), axis=0)
    
    def nuclei_vals(self):
        flat_img = self.gray_image.reshape(-1)
        flat_msk = self.combined_mask.reshape(-1).astype('bool')
        return flat_img[flat_msk], flat_img[~flat_msk]
        
    def gaussian_mixture(self):
        self.mm = sklearn.mixture.GaussianMixture(n_components=2,init_params = 'random', verbose=True)
        self.mm.fit(self.gray_image.reshape(-1)[:, np.newaxis])
        self.means = [mm.means_[0][0], mm.means_[1][0]]
        print(self.means)
        self.covariances = [mm.covariances_[0][0][0], mm.covariances_[1][0][0]]
    
    def kmeans(self):
        self.km = sklearn.cluster.KMeans(n_clusters=2, random_state=0).fit(self.gray_image.reshape(-1)[:, np.newaxis])

    def predict_kmeans(self):
        self.predictions = self.km.predict(self.gray_image.reshape(-1)[:, np.newaxis])
        if np.sum(self.predictions) > (len(self.predictions)/2.0):
            self.predictions = -(self.predictions-1)
        self.predictions = self.predictions.reshape(self.image_shape[:2])
        return self.predictions
    
    def return_string_rep(self):
        arr = []
        pixel_num = 0
        run_length = 0
        for col in range(self.predictions.shape[1]):
            for row in range(self.predictions.shape[0]):
                pixel_num += 1
                if self.predictions[row,col] == 1:
                    run_length += 1
                else:
                    if run_length > 0:
                        arr.append(str(pixel_num-run_length))
                        arr.append(str(run_length))
                        run_length = 0
        return " ".join(arr)
    
    def get_csv_line(self):
        return self.image_id + ',' + self.return_string_rep()
                    

# Training and Testing

In [8]:
training_examples = []
keys = sorted([key for key in id_to_image.keys()])
for key in keys:
    training_examples.append(Example(id_to_image[key]['images'][0]))

In [9]:
for i in range(len(training_examples)):
    if i % 20 == 0:
        print(i)
    training_examples[i].kmeans()
    training_examples[i].predict_kmeans()

0
20
40
60


In [12]:
print(len(training_examples))

65


In [14]:
f = open('submission.csv','w')
f.write('ImageId,EncodedPixels\n')
for i in range(len(training_examples)):
    print(i)
    f.write(training_examples[i].get_csv_line())
    f.write('\n')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64


In [15]:
print(keys)

['0114f484a16c152baa2d82fdd43740880a762c93f436c8988ac461c5c9dbe7d5', '0999dab07b11bc85fb8464fc36c947fbd8b5d6ec49817361cb780659ca805eac', '0a849e0eb15faa8a6d7329c3dd66aabe9a294cccb52ed30a90c8ca99092ae732', '0e132f71c8b4875c3c2dd7a22997468a3e842b46aa9bd47cf7b0e8b7d63f0925', '0ed3555a4bd48046d3b63d8baf03a5aa97e523aa483aaa07459e7afa39fb96c6', '0f1f896d9ae5a04752d3239c690402c022db4d72c0d2c087d73380896f72c466', '1747f62148a919c8feb6d607faeebdf504b5e2ad42b6b1710b1189c37ebcdb2c', '17b9bf4356db24967c4677b8376ac38f826de73a88b93a8d73a8b452e399cdff', '1879f4f4f05e2bada0ffeb46c128b8df7a79b14c84f38c3e216a69653495153b', '191b2b2205f2f5cc9da04702c5d422bc249faf8bca1107af792da63cccfba829', '1962d0c5faf3e85cda80e0578e0cb7aca50826d781620e5c1c4cc586bc69f81a', '1cdbfee1951356e7b0a215073828695fe1ead5f8b1add119b6645d2fdc8d844e', '1d9eacb3161f1e2b45550389ecf7c535c7199c6b44b1c6a46303f7b965e508f1', '1ef68e93964c2d9230100c1347c328f6385a7bc027879dc3d4c055e6fe80cb3c', '259b35151d4a7a5ffdd7ab7f171b142db8cfe40beeee67

In [18]:
training_examples[-2].get_csv_line()

'fac507fa4d1649e8b24c195d990f1fc3ca3633d917839e1751a9d412a14ab5e3,68 7 321 8 576 9 832 9 1088 9 1344 9 1599 10 1855 10 2112 8 2367 9 2623 1 2629 2 15108 1 15363 5 15619 5 15873 8 16129 8 16364 5 16385 8 16619 7 16641 8 16875 5 16897 8 16937 9 17131 6 17153 8 17192 11 17385 8 17409 8 17448 11 17640 9 17665 4 17704 10 17896 9 17921 3 17960 10 18152 10 18216 10 18408 10 18472 10 18664 11 18689 3 18729 8 18922 9 18945 3 18985 8 19178 8 19201 3 19242 6 19435 1 19439 2 19457 3 19499 3 19713 5 19969 5 20225 5 20481 5 20737 5 20993 4 21249 4 21506 1 23561 1 23815 6 24069 9 24323 11 24579 12 24835 12 25091 11 25348 10 25605 8 25861 6 26118 4 26375 1 31719 4 31780 4 31974 5 32035 6 32229 8 32256 1 32291 8 32484 11 32510 3 32546 9 32740 10 32765 4 32802 10 32996 11 33021 4 33058 10 33252 10 33267 3 33277 4 33315 9 33508 8 33523 5 33533 4 33572 8 33765 6 33775 9 33789 4 33828 8 34023 3 34031 11 34046 3 34084 7 34287 11 34302 3 34344 1 34543 12 34559 2 34799 11 35055 11 35311 10 35569 5 35575 2 358